In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **word2vec 개선-1**

앞 장의 CBOW 모델에서 어휘가 100만 개, 은닉층의 뉴런이 100개인 CBOW 모델을 생각해보면, 입력층과 출력층에는 각 100만개의 뉴런이 존재한다. 이 수많은 뉴런 때문에 중간 계산에 많은 시간이 소요된다. 정확히는 다음의 두 계산이 병목이 된다.

* 입력층의 원핫 표현과 가중치 행렬 $W_{in}$의 곱 계산
* 은닉층과 가중치 행렬 $W_{out}$의 곱 및 Softmax 계층의 계산

첫 번째는 입력층의 원핫 표현과 관련한 문제이다. 단어를 원핫 표현으로 다루기 때문에 어휘 수가 많아지면 원핫 표현의 벡터 크기도 커진다. 상당한 메모리를 차지하게 된다는 말이다. 이 문제는 Embedding 계층을 도입하는 것으로 해결한다.

두 번째는 은닉층 이후의 계산 문제이다. 우선 은닉층과 가중치 행렬 $W_{out}$의 곱만 해도 계산량이 상당하다. 그리고 Softmax 계층에서도 다루는 어휘가 많아짐에 따라 계산량이 증가하는 문제가 있다. 이 문제는 네거티브 샘플링이라는 새로운 손실 함수를 도입해 해결한다.

## **Embedding 계층**

원핫 표현에서 결과적으로 수행하는 일은 단지 행렬의 특정 행을 추출하는 것 뿐인데, 앞에서의 Matmul 계층의 행렬 곱은 거대한 벡터와 가중치 행렬을 곱한다. 원핫 표현으로의 변환과 MatMul 계층의 행렬 곱 계산은 사실 필요가 없는 것이다.

그러면 가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'을 추출하는 계층을 만들어보자. 그 계층을 Embedding 계층이라고 부른다. Embedding이란, Embedding 계층에 단어 임베딩(분산 표현)을 저장하는 것이다.

참고로, 통계 기반 기법으로 얻은 단어 벡터는 영어로 distributional representation이라고 하고, 신경망을 사용한 추론 기반 기법으로 얻은 단어 벡터는 distributed representation이라고 한다. 이 책에서는 둘 다 '분산 표현'으로 번역한다.

## **Embedding 계층 구현**

In [2]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [3]:
W[2] # 단지 원하는 행을 명시하기만 하면 끝

array([6, 7, 8])

In [4]:
idx = np.array([1, 0, 3, 0]) # 원하는 행 번호들을 명시하기만 하면 끝
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

Embedding 계층의 forward() 메서드를 구현해보자.

In [5]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    # Embedding 계층의 backward() 메서드를 구현해보자. 순전파와 마찬가지로 원하는 행에만 기울기를 전달한다.
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0

        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]      # 인덱스가 중복되었을 경우 먼저 쓰여진 값을 덮어쓰는 문제를 방지하기 위해 '할당'이 아닌 '더하기'를 한다.
        # 혹은
        # np.add.at(dW, self.idx, dout)
        # 넘파이 내장 메서드를 사용하면 효율이 훨씬 좋아진다.

        return None

이상으로 word2vec (CBOW 모델)의 구현은 입력 측 MatMul 계층을 Embedding 계층으로 전환하여 메모리 사용량을 줄이고 쓸데없는 계산도 생략할 수 있게 되었다.

# **word2vec 개선-2**

은닉층 이후의 처리(행렬 곱과 Softmax 계층의 계산)문제를 해소 하기 위하여 **네거티브 샘플링(부정적 샘플링)**이라는 기법을 사용한다.

Softmax 대신 네거티브 샘플링을 이용하면 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제할 수 있다.

## **은닉층 이후 계산의 문제점**

Embedding 계층을 도입하여 입력층 계산에서의 낭비를 줄였지만, 은닉층 이후의 처리에서 문제가 있다. 은닉층 이후에서 계산이 오래 걸리는 곳은 다음의 두 부분이다.

* 은닉층의 뉴런과 가중치 행렬($W_{out}$)의 곱
* Softmax 계층의 계산

첫 번째는 거대한 행렬을 곱하는 문제이다. 큰 행렬을 계산하려면 시간이 오래걸리고 메모리도 많이 필요하고, 역전파 때도 같은 계산을 수행하기 때문에 이 행렬 곱을 '가볍게' 만들어야 한다.

두 번째로, Softmax에서도 같은 문제가 발생한다. 어휘가 100만개 일때의 Softmax 수식을 살펴보자.

$$ y_k = \frac{exp(s_k)}{\sum_{i=1}^{1000000} exp(s_i)} $$

이와 같이 분모의 값을 얻으려면 exp 계산을 100만 번 수행해야 한다. 이 계산도 어휘 수에 비례해 증가하므로 Softmax를 대신할 '가벼운' 계산이 절실하다.

## **다중 분류에서 이진 분류로**

네거티브 샘플링 기법이란, 간단히 말해서 '다중 분류(multi-classification)'를 '이진 분류(binary-classification)'로 근사하는 것이다.

예를 들어, "맥락이 'you'와 'goodbye'일 때, 타깃 단어는 무엇입니까?"라는 질문이 아니라, "맥락이 'you'와 'goodbye'일 때, 타깃 단어는 'say' 입니까?" 라는 질문에 답하는 신경망을 생각해내는 것이다.

이렇게 하려면 출력층에는 뉴런을 하나만 준비하면 된다. 즉 출력층의 뉴런이 "say"의 점수를 출력한다.

## **시그모이드 함수와 교차 엔트로피 오차**

이진 분류 문제를 신경망으로 풀려면 점수에 시그모이드 함수를 적용해 확률로 변환하고, 손실을 구할 때는 손실 함수로 '교차 엔트로피 오차'를 사용한다. 이 둘은 이진 분류 신경망에서 가장 흔하게 사용하는 조합이다. (다중 분류의 경우, 출력층에서 시그모이드 함수가 아닌 '소프트 맥스 함수'를 사용한다.)

시그모이드 함수의 출력($y$)는 소프트맥스 함수의 출력과 마찬가지로 '확률'로 해석할 수 있다. 시그모이드 함수에 사용되는 손실함수 '교차 엔트로피 오차'는 다음과 같이 쓸 수 있다.

$$ L = -(tlogy + (1-t)log(1-y)) $$

$t$가 1이면 $-logy$가 출력되고, 반대로 $t$가 0이면 $-log(1-y)$가 출력된다.
또한 소프트 맥스와 마찬가지로 역전파의 값은 $y-t$, 즉 정답과 출력의 오차이다.

## **다중 분류에서 이진 분류로 (구현)**

우선, 앞서 설명한 Embedding 계층과 'dot 연산(내적)'의 처리를 합친 계층인 Embedding Dot 계층을 도입하자. 그 구현은 다음과 같다.

In [6]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None       # 순전파 시의 계산 결과를 잠시 유지하기 위한 변수로 사용

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

## **네거티브 샘플링**

지금까지 배운 것으로 주어진 문제를 '다중 분류'에서 '이진 분류'로 변환할 수 있었지만, 긍정적인 예(정답)에 대해서만 학습했기 때문에 부정적인 예(오답)를 입력하면 어떤 결과가 나올지 확실하지 않다.

우리가 하고싶은 일은 긍정적 예("say")에 대해서는 Sigmoid 계층의 출력을 1에 가깝게 만들고, 부정적 예("say"이외의 단어)에 대해서는 Sigmoid 계층의 출력을 0에 가깝게 만드는 것이다. 그리고 이런 결과를 만들어주는 가중치가 필요하다.

모든 부정적 예를 대상으로 하여 이진 분류를 학습시키면, 어휘 수 증가에 대처하는 것이 목적인 것에 반한다. 그래서 근사적인 해법으로, 부정적 예를 몇 개(선택하는 방법은 뒤에서 다룬다) 선택한다. 즉, 적은 수의 부정적 예를 샘플링 해 사용한다.

이것이 바로 '네거티브 샘플링' 기법이 의미하는 바이다.

정리하면, 네거티브 샘플링 기법은 긍정적 예를 타깃으로 한 경우의 손실을 구하고, 동시에 부정적 예를 몇 개 샘플링하여, 그 부정적 예에 대해서도 마찬가지로 손실을 구한다. 그리고 각자의 데이터의 손실을 더한 값을 최종 손실로 한다.

## **네거티브 샘플링의 샘플링 기법**

부정적 예를 어떻게 샘플링 하냐는 질문에 답하자면, 말뭉치의 통계 데이터를 기초로 샘플링하면 된다. 말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출하는 것이다.

말뭉치에서의 단어별 출현 횟수를 바탕으로 확률분포를 구한 다음, 그 확률분포에 따라서 샘플링을 수행하기만 하면 된다. 따라서 '희소한 단어'는 선택되기가 어렵다.

우연히 '희소한 단어'만 선택하면 결과도 나빠질 것이다. 실전 문제에서도 희소한 단어는 거의 출현하지 않기 때문에, 드문 단어를 잘 처리하는 일은 중요도가 낮다. 그보다 흔한 단어를 잘 처리하는 편이 좋은 결과를 낳는다.

확률분포에 따라 샘플링하는 예를 파이썬 코드로 확인해보자.

In [7]:
# 0에서 9까지의 숫자 중 하나를 무작위로 샘플링
np.random.choice(10)

7

In [8]:
np.random.choice(10)

2

In [9]:
# words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'I'

In [10]:
# 5개만 무작위로 샘플링(중복 있음)
np.random.choice(words, size=5)

array(['say', 'you', 'hello', 'hello', '.'], dtype='<U7')

In [11]:
# 5개만 무작위로 샘플링(중복 없음)
np.random.choice(words, size=5, replace=False)

array(['I', 'hello', 'you', '.', 'goodbye'], dtype='<U7')

In [12]:
# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'say'

word2vec의 네거티브 샘플링에서는 기본 확률분포에 0.75를 제곱한다.

$$P^{'}(w_i)=\frac{P(w_i)^{0.75}}{\sum_{j}^n P(w_j)^{0.75}}$$

그 이유는 출현 확률이 낮은 단어를 '버리지 않기' 위해서이다. 정확히 말해서, '0.75 제곱'을 함으로써, 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있다. 예를 보자.

In [13]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


이전엔 확률이 0.01이던 원소가 수정 후에는 0.0265.. 가 되었다. 낮은 확률의 단어가 (조금 더)쉽게 샘플링되도록 하기 위한 구제 조치로써 '0.75제곱'을 수행한다. 0.75라는 수치에는 이론적인 의미는 없으니 다른 숫자로 설정해도 된다.

이 처리를 담당하는 UnigramSampler 클래스를 구현하는데, 자세한 설명은 하지 않겠다.

In [14]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

## **네거티브 샘플링 구현**

In [15]:
class NegativeSamplingLoss:

    # 초기화 메서드
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    # 순전파 구현
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    # 역전파 구현
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

# **개선판 word2vec 학습**

## **CBOW 모델 구현**

In [16]:
path = '/content/drive/MyDrive/밑바닥부터시작하는딥러닝2/deep-learning-from-scratch-2-master'
import sys
sys.path.append(path)
from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

## **CBOW 모델 학습 코드**

In [25]:
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from ch04.skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)

if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

버전 호환때문에 오류가 뜨므로 pickle을 이용해보자.

## **CBOW 모델 평가**

In [23]:
from common.util import most_similar, analogy
import pickle

pkl_file = path + '/ch04/cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


결과를 보면 CBOW 모델로 획득된 단어의 분산 표현은 제법 괜찮은 특성을 지닌다고 말할 수 있다.

word2vec으로 얻은 단어의 분산 표현은 비슷한 단어를 가까이 모을 뿐 아니라, 더 복잡한 패턴을 파악하는 것으로 알려져 있다. 예를 들어 "king - man + woman = queen"으로 유명한 유추 문제이다. 정확하게 말하면, word2vec의 단어의 분산 표현을 사용하면 유추 문제를 벡터의 덧셈과 뺄셈으로 풀 수 있다는 뜻이다.

In [24]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875


단수형과 복수형을 올바르게 파악하고, 비교급 단어등을 제시하는 것을 보면 단어의 단순한 의미뿐 아니라 문법적인 패턴도 파악할 수 있음을 알 수 있다.

## **word2vec을 사용한 애플리케이션의 예**

word2vec으로 얻은 단어의 분산 표현은 비슷한 단어를 찾는 용도로 이용할 수 있다. 그러나 장점은 여기서 끝나는 것이 아니다. 자연어 처리 분야에서 단어의 분산 표현이 중요한 이유는 **전이 학습(transfer learning)**에 있다. 전이 학습은 한 분야에서 배운 지식을 다른 분야에도 적용하는 기법이다.

자연어 문제를 풀 때 word2vec의 단어 분산 표현을 처음부터 학습하는 일은 거의 없다. 먼저 큰 말뭉치(위키백과 등)으로 학습을 끝난 후, 그 분산 표현을 각자의 작업에 이용한다.

단어의 분산 표현은 단어를 고정 길이 벡터로 변환해준다는 장점도 있다. 문장 또한 단어의 분산 표현을 사용하여 고정 길이 벡터로 변환할 수 있다. 이 중 가장 간단한 방법은 문장의 각 단어를 분산 표현으로 변환하고 그 합을 구하는 bag-of-words 가 있다. 이 모델은 순서를 고려하지 않는다.

단어나 문장을 고정 길이로 변환해주는 것이 장점이 되는 이유는 자연어를 벡터로 변환할 수 있다면 일반적인 머신러닝 기법(신경망이나 SVM 등)을 적용할 수 있기 때문이다.

이를 통해 감성분석 등의 애플리케이션이 가능하다.

## **단어 벡터 평가 방법**

단어의 분산 표현이 어떻게 평가할까에 대해서 논해보자.

단어의 분산 표현은 애플리케이션에서 사용되는 것이 대부분이므로, 궁극적으로 정확도 높은 시스템을 구축해야한다. 그 시스템은 여러 시스템으로 구성되는 것을 생각해보아야 한다. 단어의 분산 표현을 만드는 시스템(word2vec)과 특정 문제에 대해 분류를 수행하는 시스템(감정을 분류하는 SVM 등)이 그 예이다.

단어의 분산 표현의 우수성은 실제 애플리케이션과는 분리해 평가하는 것이 일반적이다(함께 하면 시간이 오래 걸릴 수 있다). 이때 자주 사용되는 평가 척도가 단어의 '유사성'이나 '유추 문제'를 활용한 평가이다.

단어의 유사성 평가에서는 사람이 작성한 단어 유사도를 검증 세트로 사용해 평가하는 것이 일반적이다. 사람이 부여한 점수와 word2vec에 의한 코사인 유사도 점수를 비교해 그 상관성을 보는것이다.

유추 문제를 활용한 평가에서는 유추 문제를 출제하고, 그 정답률로 단어의 분산 표현의 우수성을 측정한다. 유추 문제를 이용하면 '단어의 의미나 문법적인 문제를 제대로 이해하고 있는지'를 어느 정도 측정할 수 있다.

하지만 유추 문제에 의한 평가가 높다고 해서 애플리케이션에서도 반드시 좋은 결과가 나오리라는 보장은 없음을 주의해야 한다.

# **정리**

* **Embedding 계층은 단어의 분산 표현을 담고 있으며, 순전파 시 지정한 단어 ID의 벡터를 추출한다.**
* **word2vec은 어휘 수의 증가에 비례하여 계산량도 증가하므로, 근사치로 계산하는 빠른 기법을 사용하면 좋다.**
* **네거티브 샘플링은 부정적 예를 몇 개 샘플링하는 기법으로, 이를 이용하면 다중 분류를 이진 분류처럼 취급할 수 있다.**
* **word2vec으로 얻은 단어의 분산 표현에는 단어의 의미가 녹아들어 있으며, 비슷한 맥락에서 사용되는 단어는 단어 벡터 공간에서 가까이 위치한다.**
* **word2vec의 단어의 분산 표현을 이용하면 유추 문제를 벡터의 덧셈과 뺄셈으로 풀 수 있게 된다.**
* **word2vec은 전이 학습 측면에서 특히 중요하며, 그 단어의 분산 표현은 다양한 자연어 처리 작업에 이용할 수 있다.**